In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, datasets
from torchinfo import summary

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('..')

from python_scripts import data_setup, engine

c:\Users\TimJimTangtong\Miniconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [3]:
train_labels = pd.read_csv('../data/COVID_19_XRAY/train/labels.csv')

In [5]:
import os
from pathlib import Path

current_path = Path('../data/COVID_19_XRAY/train')
covid_path = Path('../data/COVID_19_XRAY/train/covid')
normal_path = Path('../data/COVID_19_XRAY/train/normal')

covid_path.mkdir(parents=True, exist_ok=True)
normal_path.mkdir(parents=True, exist_ok=True)

for dirpath, dirnames, filenames in os.walk(current_path):
  if dirpath == str(current_path):
    image_names = filenames

image_names.sort(key=lambda x: (len(x), x))
image_names.pop(0)

print(image_names)

['image_001.png', 'image_002.png', 'image_003.png', 'image_004.png', 'image_005.png', 'image_006.png', 'image_007.png', 'image_008.png', 'image_009.png', 'image_010.png', 'image_011.png', 'image_012.png', 'image_013.png', 'image_014.png', 'image_015.png', 'image_016.png', 'image_017.png', 'image_018.png', 'image_019.png', 'image_020.png', 'image_021.png', 'image_022.png', 'image_023.png', 'image_024.png', 'image_025.png', 'image_026.png', 'image_027.png', 'image_028.png', 'image_029.png', 'image_030.png', 'image_031.png', 'image_032.png', 'image_033.png', 'image_034.png', 'image_035.png', 'image_036.png', 'image_037.png', 'image_038.png', 'image_039.png', 'image_040.png', 'image_041.png', 'image_042.png', 'image_043.png', 'image_044.png', 'image_045.png', 'image_046.png', 'image_047.png', 'image_048.png', 'image_049.png', 'image_050.png', 'image_051.png', 'image_052.png', 'image_053.png', 'image_054.png', 'image_055.png', 'image_056.png', 'image_057.png', 'image_058.png', 'image_059.pn

In [6]:
for i in range(len(train_labels)):
  if train_labels.label.iloc[i] == 'covid':
    os.rename(current_path / image_names[i], covid_path / image_names[i])
  else:
    os.rename(current_path / image_names[i], normal_path / image_names[i])

In [7]:
def walk_through_dir(dir_path):
    '''Walks through dir_path returning its contents.'''
    for dirpath, dirnames, filenames in os.walk(dir_path):
        num_images = len(list(Path(dirpath).glob('*.png')))
        print(f'There are {len(dirnames)} directories and {num_images} images in {dirpath}.')

walk_through_dir(current_path)

There are 2 directories and 0 images in ..\data\COVID_19_XRAY\train.
There are 0 directories and 1000 images in ..\data\COVID_19_XRAY\train\covid.
There are 0 directories and 1000 images in ..\data\COVID_19_XRAY\train\normal.


In [8]:
whole_dataset = datasets.ImageFolder(
  root=current_path,
  transform=transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(
      [0.485, 0.456, 0.406],
      [0.229, 0.224, 0.225]
    )
  ])
)

# 모델 비교 단계에서 사용
# train_dataset, valid_dataset = data_setup.split_dataset(
#     dataset=whole_dataset,
#     split_size=0.8,
#     seed=42
# )

# 최종 제출을 위해 모든 이미지 사용하여 학습
train_dataset, _ = data_setup.split_dataset(
    dataset=whole_dataset,
    split_size=1,
    seed=42
)

_, valid_dataset = data_setup.split_dataset(
    dataset=whole_dataset,
    split_size=0.2,
    seed=42
)

class_names = whole_dataset.classes

[INFO] Splitting dataset of length 2000 into splits of size: 2000 and 0
[INFO] Splitting dataset of length 2000 into splits of size: 400 and 1600


In [9]:
learning_rate_list = [1e-4, 1e-5, 1e-6] # 각 LR 별로 50 epoch 씩 연달아 학습 진행
weight_decay_list = [1e-4]
epochs_list = [50]
batch_size_list = [16]

In [10]:
def model_generator(weights=None):
    weights = torchvision.models.ResNet152_Weights.DEFAULT
    model = torchvision.models.resnet152(weights=weights)
    model.fc = nn.Linear(
        in_features=2048,
        out_features=len(class_names),
        bias=True
    )
    return model

model = model_generator()

tuning_results = engine.HP_tune_train(
    model=model,
    model_generator=None,
    model_weights=None,
    model_name='ResNet152_covid-xray_2000',
    train_dataset=train_dataset,
    test_dataset=valid_dataset,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_tensorboard_writer=True,
    device=device,
    gradient_accumulation_num=1
)

[INFO] Creating SummaryWriter saving to ../runs\2023-04-04\ResNet152_covid-xray_2000_test\ResNet152_covid-xray_2000\LR_0.0001_WD_0.0001_EP_50_BS_16_GA_1


  2%|▏         | 1/50 [01:10<57:30, 70.43s/it]

Epoch: 0 | Train_loss: 0.5036, Train_acc: 0.7555 | Test_loss: 0.4153, Test_acc: 0.8163


  4%|▍         | 2/50 [02:19<55:42, 69.63s/it]

Epoch: 1 | Train_loss: 0.3971, Train_acc: 0.8165 | Test_loss: 0.3694, Test_acc: 0.8363


...

Epoch: 47 | Train_loss: 0.0620, Train_acc: 0.9760 | Test_loss: 0.0457, Test_acc: 0.9825


 98%|█████████▊| 49/50 [55:23<01:07, 67.62s/it]

Epoch: 48 | Train_loss: 0.0431, Train_acc: 0.9835 | Test_loss: 0.0430, Test_acc: 0.9812


100%|██████████| 50/50 [56:31<00:00, 67.82s/it]

Epoch: 49 | Train_loss: 0.0412, Train_acc: 0.9880 | Test_loss: 0.0340, Test_acc: 0.9881


In [11]:
print(pd.DataFrame(tuning_results))

   learning_rate  weight_decay  epochs  batch_size   train_loss    test_loss  \
0       0.000100        0.0001      50          16   0.14178865   0.10487087   
1       0.000010        0.0001      50          16  0.061279908  0.045043487   
2       0.000001        0.0001      50          16  0.041209783  0.034047224   

  train_acc  test_acc  
0     0.945    0.9575  
1      0.97  0.979375  
2     0.988  0.988125  


In [12]:
torch.save(model.state_dict(), '../models/corona-xray/230404_ResNet152_2000.pth')

In [16]:
test_path = Path('../data/COVID_19_XRAY/test')

for dirpath, dirnames, filenames in os.walk(test_path):
  if dirpath == str(test_path / 'test'):
    image_names = filenames

image_names.sort(key=lambda x: (len(x), x))

In [17]:
test_dataset = datasets.ImageFolder(
    test_path,
    transform=transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
        )
    ])
)

In [18]:
test_pred = []

model.eval()
with torch.inference_mode():
  for i in range(400):
      test_pred.append(class_names[torch.argmax(model(test_dataset[i][0].unsqueeze(0).to(device)))])

output = pd.DataFrame({'filename': image_names, 'label': test_pred})
output.to_csv('submission.csv', index=False)
print('Submission saved!')

Submission saved!
